## Install PUDL
* Until we get our custom Docker image built, PUDL needs to be installed in your user environment each session.
* If you are using this notebook on the Catalyst JupyterHub, and this is the first notebook you've used this session, then uncomment the commands in the following cell and run it before anything else.

In [ ]:
#!conda install --yes --quiet python-snappy
#!pip install --upgrade pip
#!pip install --quiet git+https://github.com/catalyst-cooperative/pudl.git@jupyterhub-beta
#!cp ~/shared/shared-pudl.yml ~/.pudl.yml

In [ ]:
# import the necessary packages
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlalchemy as sa
import pudl

# Connecting to the PUDL Databases

This notebook will walk you through several ways of pulling data out of the Public Utility Data Liberation (PUDL)
project databases and into [Pandas](https://pandas.pydata.org/) Dataframes for analysis and visualization.

This notebook assumes you have a development version of the [PUDL Python package](https://github.com/catalyst-cooperative/pudl) installed, and a complete PUDL database available locally, in the location expected by the Python package.

If you have any questions or feedback you can:
* [Create an issue](https://github.com/catalyst-cooperative/pudl-tutorials/issues) in the GitHub repo for our tutorials, or
* Contact the team at: pudl@catalyst.coop

## Direct SQLite Access
Much of the PUDL data is published as [SQLite database files](https://www.sqlite.org/index.html). These are relational databases generally intended for use by a single user at a time. If you're already familiar with databases and SQL in Python, you can access them just like you would any other. [Support for SQLite](https://docs.python.org/3/library/sqlite3.html) is built into the Python standard libraries, and the popular [SQLAlchemy](https://www.sqlalchemy.org) Python package also has extensive support for SQLite.  Here's one in-depth resource on using Python, SQLite and SQLAlchemy together: [Data Management with Python, SQLite, and SQLAlchemy](https://realpython.com/python-sqlite-sqlalchemy/)

For the rest of these tutorials, we're going to assume you want to get the data into Pandas as quickly as possible for interactive work.


## Database Normalization
The data in the PUDL database has been extensively deduplicated, [normalized](https://en.wikipedia.org/wiki/Database_normalization) and generally organized according to best practices of [tidy data](https://tidyr.tidyverse.org/articles/tidy-data.html) in order to ensure that it is internally self-consistent and free of errors. As a result, you'll often need to combine information from more than one table to make it readable or to get all the information you need for your analysis in one place. We've built some tools to do this automatically, which we'll get to below.

## Locate the PUDL DB file
Each SQLite database is stored within a single file. To access the data, you need to know where that file is. With the location of the file, you can create an [SQLAlchemy connection engine](https://docs.sqlalchemy.org/en/13/core/engines.html), which Pandas will use to read data out of the database. PUDL stores its data in a directory structure generally organized by file format. We store the paths to those directories and the SQLAlchemy database URLs in a Python dictionary that's usually called `pudl_settings`. Note that  a URL is just a path to a file that could be either local (on your computer) or remote (on someone else's computer). The following command will construct that `pudl_settings` dictionary based on some directory paths stored in the `.pudl.yml` file in your home directory. Printing out the dictionary contents you can see where PUDL will look for various resources.

In [ ]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_settings

## The SQLAlchemy Connection Engine
The `sqlalchemy.create_engine()` function takes a database URL and creates an Engine that knows how to interact with the database. It can do things like list out the names of all the tables in the database.

In [ ]:
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
# see all the tables inside of the database
pudl_engine.table_names()

# Reading data with `pandas.read_sql()`
The [pandas.read_sql()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html) method is the simplest way to pull data from an SQL database into a dataframe. You can give it an SQL statement to execute, or just the name of a table to read in its entirety.

## Read a whole table
Reading an entire table all at once is easy. It isn't very memory efficient but there's less than 1 GB of data in the PUDL database, so in most cases this is a fine option. Once you've had a chance to poke around at the whole table a bit, you can select the data that's actually of interest out of it for your analysis or visualization.

You can also explore the contents of the database interactively online at https://data.catalyst.coop if you want to familiarize yourself with its contents in a more graphical way first.

In [ ]:
generation_df = pd.read_sql("generation_eia923", pudl_engine)
generation_df.info(memory_usage="deep")

In [ ]:
generation_df.sample(10)

## Select specific data using SQL
If you're familiar with SQL, and you already know what subset of the data you want to pull out of the database, you can give Pandas an SQL statement directly, along with the `pudl_engine`, and it will put the results of the SQL statement into a dataframe for you.

For example, the following statement sums the nameplate capacities of generators by power plant, for every generator that reported a capacity in the EIA 860 in 2019, excluding those in Alaska and Hawaii. It sorts the results by capacity with the biggest plants first, and only returns the biggest 1000 plants.

[Compare with the results from our online database](https://data.catalyst.coop/pudl?sql=select%0D%0A++plants.plant_id_eia%2C%0D%0A++plants.plant_name_eia%2C%0D%0A++SUM%28gens.capacity_mw%29+as+plant_capacity_mw%2C%0D%0A++latitude%2C%0D%0A++longitude%0D%0Afrom%0D%0A++generators_eia860+as+gens%0D%0Ajoin%0D%0A++plants_entity_eia+as+plants%0D%0Awhere%0D%0A++plants.plant_id_eia+%3D+gens.plant_id_eia%0D%0A++and+gens.report_date+%3D+%222019-01-01%22%0D%0A++and+plants.state+not+in+%28%22HI%22%2C+%22AK%22%29%0D%0Agroup+by%0D%0A++plants.plant_id_eia%0D%0Aorder+by%0D%0A++plant_capacity_mw+desc).

This method is much faster and less memory intensive than reading whole tables, but it requires familiarity with SQL and the structure of the database. If you have a solid state disk and plenty of RAM, reading whole tables into memory is generally plenty fast, and shouldn't run into memory constraints.

In [ ]:
example_sql = """
SELECT
  plants.plant_id_eia,
  plants.plant_name_eia,
  SUM(gens.capacity_mw) AS plant_capacity_mw,
  latitude,
  longitude
FROM
  generators_eia860 AS gens
JOIN
  plants_entity_eia AS plants
WHERE
  plants.plant_id_eia = gens.plant_id_eia
  AND gens.report_date = "2019-01-01"
  AND plants.state not in ("HI", "AK")
GROUP BY
  plants.plant_id_eia
ORDER BY
  plant_capacity_mw DESC
LIMIT 1000;
"""
big_plants_df = pd.read_sql(example_sql, pudl_engine)
big_plants_df

## The SQLAlchemy expression language
SQLAlchemy provides a Python API for building complex SQL queries, and `pandas.read_sql()` can accept these query objects in place of the SQL statement written out by hand as above. [See the SQLAlchemy documentation for more details](https://docs.sqlalchemy.org/en/13/core/tutorial.html).

# Read tables using the PUDL output layer
Early on in the development of the PUDL database, we found that we were frequently joining the same tables together, and calculating the same derived values in Pandas during our interactive analyses. So we wrote some code to do that work automatically and uniformly. We call this the PUDL Output Layer. It brings in fields like plant and utility names from their home tables, so you have more than just the numeric ID to go by, caches dataframes internally for re-use, and can do some time series aggregation.

These outputs are "denormalized" -- meaning that data will be duplicated in different output tables, and they will contain derived values that don't represent unique information. This structure isn't good inside a database, but it's great for interactive use.

The 2nd notebook in this tutorial is all about the `PudlTabl` objects, which we usually name `pudl_out`, but here is a quick preview.

If you want to access de-normalized tables, we've built an access methodology that saves access methods for most denormalized tables in PUDL and analysis build ontop of PUDL tables. There is a whole other notebook that covers the output tables so if you want more info on that.

## Create a PudlTabl output object
The tabular output object needs to know what PUDL database it's connecting to (via the `pudl_engine` argument), and optionally, what time frequency it should aggregate tables on.

In [ ]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine)

## Construct denormalized dataframes
The `PudlTabl` object, called `pudl_out` here, has a bunch of methods corresponding to individual tables within the database. They typically use abbreviated names. Hitting `Tab` will show you a preview the available methods.

The `gen_eia923()` method corresponds to the `generation_eia923` table in the database, which details the monthly net generation from each generator reporting on the EIA Form 923.

Note: if you re-run the cell, it will complete almost instantly, because the dataframe has been cached inside the `pudl_out` object for later use.

In [ ]:
%%time
gen_eia923 = pudl_out.gen_eia923()
gen_eia923.info()

In [ ]:
gen_eia923.sample(10)

## Compare with the normalized DB table
The denormalized version of the table above includes fields like `utility_name_eia923` and `plant_name_eia923` and `plant_id_pudl` which are all useful, but aren't fundamentally part of this table -- they can all be looked up in other tables based on the value of `plant_id_eia` found in the original `generation_eia923` table, so storing them in this table would mean duplicating data.  You can see what the original table looks like below.

Note also that since we're going back to the database directly rather than accessing the cached dataframe within the `pudl_out` object, this query will take a few seconds to run, just like the first time we read the table using `pudl_out` above.

In [ ]:
%%time
gen_eia923_normalized = pd.read_sql("generation_eia923", pudl_engine)
gen_eia923_normalized.info()

In [ ]:
gen_eia923_normalized.sample(10)

# FERC Form 1: Here Be Dragons
You might have noticed up above that there were actually two SQLite database URLs in the `pudl_settings` object... One for PUDL, and another for FERC Form 1.

In [ ]:
pudl_settings

## FERC Form 1: Direct vs. PUDL
The PUDL database contains a tiny fraction of the data available in the original FERC Form 1 -- we have only taken the time to clean a handful of the FERC tables. The original FERC Form 1 data is often very messy and poorly organized. However, if you need to access one of the original 113 tables that we haven't integrated yet, they're all available, going back to 1994. The original tables are only accessible via direct queries (either using SQL or pulling whole tables) from the original FERC Form 1 database, so you'll have to use the `pandas.read_sql()` methods outlined above.

If there are particular tables within the FERC Form 1 that you think are important to get cleaned up, let us know so we can prioritize them going forward!

In [ ]:
ferc1_engine = sa.create_engine(pudl_settings["ferc1_db"])
# see all the tables inside of the database
ferc1_engine.table_names()